In [ ]:
import matplotlib.pyplot as plt

%matplotlib inline

from functools import partial

import einops
import librosa
import numpy as np
import soundfile
import torch
import torchaudio
import torchaudio.functional as AF
from IPython.display import Audio
from torch.nn import functional as F
from torchaudio.transforms import Resample

import myddsp.constants as C
import myddsp.preprocessors as pre

In [ ]:
def plot(x, figsize=(12, 6), *args, **kwargs):
    fig, ax = plt.subplots(figsize=figsize)
    ax.plot(x, *args, **kwargs)


def matshow(x, figsize=(50, 50), *args, **kwargs):
    fig, ax = plt.subplots(figsize=figsize)
    ax.matshow(x, *args, origin="lower", **kwargs)

## Load example

In [ ]:
y, _ = librosa.load("../data/test/samples/violin-4.wav", sr=48000, mono=True)
Audio(y, rate=48000)

In [ ]:
yy = torch.from_numpy(y)
yy = yy.unsqueeze(0).unsqueeze(0)

## Explore loudness calculation methods

In [ ]:
C.N_FFT, C.HOP_LENGTH, C.CREPE_HOP_LENGTH

In [ ]:
ld = pre.Loudness()
ld = ld.eval()
for p in ld.parameters():
    p.requires_grad = False

In [ ]:
with torch.inference_mode():
    amps = ld(yy).flatten().numpy() / 100 + 1

In [ ]:
q = np.round(amps * 152).astype("int")
kq = np.take(np.eye(152), q, axis=0)
matshow(kq.T)

In [ ]:
kq.shape

## Explore pitch calculation methods

In [ ]:
crepe = pre.F0("full").cuda()
crepe = crepe.eval()
for p in crepe.parameters():
    p.requires_grad = False

In [ ]:
rs = Resample(48000, 16000)

In [ ]:
with torch.inference_mode():
    acti = crepe(rs(yy).cuda())

In [ ]:
matshow(acti.cpu().T)

In [ ]:
const = 9
delta = 0.01
peak_part = partial(
    librosa.util.peak_pick,
    pre_max=const,
    post_max=const,
    pre_avg=const,
    post_avg=const * 2,
    delta=delta,
    wait=const * 3,
)
result = list(map(peak_part, acti.cpu().numpy()))

pitch_condition = np.zeros_like(acti.cpu())
no_pitch = 0
for idx, r in enumerate(result):
    try:
        pitch_condition[idx, r] = acti[idx, r].cpu()
    except IndexError:
        no_pitch += 1
        continue

print(f"{no_pitch} frames have no pitch!")

matshow(pitch_condition.T)